# Minesweeper AI

The goal of this project is to create an AI that can solve the game Minesweeper using inference and propositional logic. This AI is fallible because the Minesweeeper game (1) has imperfect information - meaning that the states of the squares are hidden - and (2) is non deterministic - meaning that there is an element of randomness.

The algorithm has the function `make_safe_move` to make a move in a list of known safe squares, and `make_random_move` if no safe moves are found.

The following functions draw inferences:

- `known_mines` - returns a list of all cells known to be mines
- `known_safes` - returns a list of all cells known to be safes
- `mark_mines` - updates a sentence given that a cell is known to be a mine.
- `mark_safes` - updates a sentence given that a cell is known to be a safe cell.

There's also the function `get_neighbors`.

The most exciting aspect of the project is `add_knowledge`. The code for it is displayed below.

### Explanation of Add_Knowledge:

There are 3 ways that the AI can draw inferences.


![infer_mines](assets/images/infer_mines.png)

(Image courtesy of Harvard University)

Given this image, the AI would know that {E, F, H} = 3.

1. Suppose {A, B, C} = 2. If we were told that C is safe, C can be removed, and {A, B} = 2 means that both A & B are mines.

2. However, if we were told that B is a mine, then {A, C} = 1: either A or C must be a mine.


![subset_inference](assets/images/subset_inference.png)

3.  Consider the above image: the AI would make 2 inferences: `{A, B, C} = 1` and `{A, B, C, D, E} = 2`. Logically, we can then infer that {D, E} = 1, because the 1st set is a subset of the 2nd set.

`add_knowledge` attempts to translate these ideas into code.

In [ ]:
    def add_knowledge(self, cell, count):
        
        self.moves_made.add(cell)
        self.safes.add(cell)

        
        #marks known mines & safes in the sentence
        new_sent = Sentence(list(self.neighbors(cell)), count)
        print("new sentence:", new_sent)

        for mine in self.mines:
            new_sent.mark_mine(mine)

        for safe in self.safes:
            new_sent.mark_safe(safe)

        new_sent = Sentence(set(new_sent.cells), new_sent.count)
        print("marked sentence", new_sent)
        
        
        #appends new sentence into knowledge
        if new_sent not in self.knowledge:
            self.knowledge.append(new_sent)
        
        
        #for existing sentences, check if new inferences (1 & 2) can be drawn
        for sentence in self.knowledge:
            
            #convert all sentences to list for removal
            sentence.cells = list(sentence.cells)
            
            for mine in sentence.known_mines():
                print("mine found", mine)
                if mine not in self.mines:
                    self.mines.add(mine)
                    
                if mine in sentence.cells:
                    print("marking mine...", mine)
                    sentence.mark_mine(mine)
                    

            for safe in sentence.known_safes():                
                if safe not in self.safes:
                    self.safes.add(safe)
                    
                if safe in sentence.cells:
                    print("marking safes...", safe)
                    sentence.mark_safe(safe)
        
        
            sentence.cells = set(sentence.cells)

            if sentence == Sentence(set(), 0) or sentence == Sentence([], 0):
                print("removing sentence...", sentence)
                self.knowledge.remove(sentence)

                
        #deals with the 3rd type of inference
        
        for sent1, sent2 in list(itertools.combinations(self.knowledge, 2)):
            sent1.cells = set(sent1.cells)
            sent2.cells = set(sent2.cells)
            print("sentence 1 & 2", sent1, sent2)
            
            
            if sent1.cells.issubset(sent2.cells):
                    
                new_sent = Sentence((sent2.cells - sent1.cells), sent2.count - sent1.count)
            elif sent2.cells.issubset(sent1.cells):
                new_sent = Sentence((sent1.cells - sent2.cells), sent1.count - sent2.count)
    
            if new_sent not in self.knowledge:
                print("appending new sentence...", new_sent)
                self.knowledge.append(new_sent)